# Lab Assignment 4: Using APIs in Python
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will work with the public API provided by [genius.com](https://genius.com/), a website that calls itself "the world’s biggest collection of song lyrics and musical knowledge." You will need to read the API documentation carefully, acquire an access key, and use it without sharing it to pull data from this API into Python. You will also practice using a library made specifically to wrap around `requests` to make calling from the Genius API easier.

## Problem 0
Import the following libraries:

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import os
import dotenv
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks

## Problem 1
The Genius API documentation is here: https://docs.genius.com/#/getting-started-h1. Read through the documentation carefully. Although the Genius API is free and public, it still requires users to use an access key to use the API. In this case, Genius provides users with three codes: a client ID, a client ID secret, and a client access token. Use the documentation to find a way to obtain these codes for yourself. Write a paragraph that describes all of the steps you needed to take (but DO NOT list your access codes in this paragraph).

Some hints and cautions: 

1. Before you can use the API, you will need a regular, free user account with Genius. Sign-up here: https://genius.com/signup_or_login

2. Genius's API is built to support third-party app development, not data scientists. The language is entirely geared toward app development. Under "Authentication" there are instructions to third-party developers for guiding their own users in getting access keys. That's not relevant to getting access for yourself. It's not hard to get an access key, but the guidance here is not very clear. Be patient and read everything in the Authentication section carefully.

3. When you arrive at the page that allows you to register for API access keys, the language is still geared toward app development. You will be prompted to name your app and provide the URLs associated with the app. It doesn't much matter what you name your app, and I just used the Collab main page (https://collab.its.virginia.edu/portal?containerLogin=true) for the URLs.

4. When you get your codes, copy them in a text file. In problem 2 you will copy these codes over again to a `.env` file. [4 points]

## Answer

To create `.env` with `client_id`, `client_secret`, and `client_access_token`, I navigated to https://docs.genius.com/. I followed the link entitled "API Client management page" to https://genius.com/api-clients. I clicked a button with text "New API Client". I entered app name "Working_With_APIs_In_Python" and app website URL https://github.com/tslever. I clicked the button with text "Save". I copied a client ID, client secret, and generated access token into `.env`.

## Problem 2
Create a `.env` file for this project. Open it, copy your access codes into it, and save it. Then use Python code to load the environmental variables contained in the `.env` file, and create variables that contain each of the three codes. (You can print these variables to make sure it worked, but do not allow your access keys to display in your notebook). [4 points]

In [2]:
dotenv.find_dotenv()

'/Users/tlever/Tom_Levers_Git_Repository/UVA/3--Practice_And_Application_Of_Data_Science/4--Working_With_APIs_In_Python/.env'

In [3]:
dotenv.load_dotenv()

True

In [4]:
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')
client_access_token = os.getenv('client_access_token')

## Problem 3
The root for all Genius APIs is https://api.genius.com. Find the endpoint for the Search API. (You will have to click the "Authorize with Genius" button in the upper-right corner if you haven't already done so). Use the `requests` library to issue a search for Bob Dylan. Genius's API is organized in a way that every individual artist has his or her own API endpoint. Display a portion of the JSON output that displays the API endpoint path for the data on Bob Dylan. 

Hint: to authenticate, specify your access token (not your client ID or client secret) as the `access_token` parameter. You will have to dig around the JSON output to find the artist ID, but it is listed under `primary_artist` several branches down the JSON tree. [4 points]

In [5]:
root = 'https://api.genius.com'
endpoint = '/search?q=Bob%20Dylan'
parameters = {
    'access_token': client_access_token
}
response = requests.get(root + endpoint, params = parameters)
text = response.text
response_body_as_JSON = json.loads(text)
subresponse = response_body_as_JSON.get('response')
hits = subresponse.get('hits')
first_hit = hits[0]
result = first_hit.get('result')
primary_artist = result.get('primary_artist')
api_path = primary_artist.get('api_path')
api_path

'/artists/181'

## Problem 4
Add `/songs` to the end of the the endpoint path you found in problem 3 and use this path to request the 20 most popular Bob Dylan songs. Organize these data in a `pandas` data frame. [4 points]

In [9]:
root = 'https://api.genius.com'
endpoint = api_path + '/songs'
parameters = {
    'access_token': client_access_token
}
response = requests.get(root + endpoint, params = parameters)
text = response.text
response_body_as_JSON = json.loads(text)
response_body_as_JSON
subresponse = response_body_as_JSON.get('response')
songs = subresponse.get('songs')
data_frame = pd.json_normalize(songs)
data_frame = pd.json_normalize(response_body_as_JSON, record_path = ['response', 'songs'])
data_frame

,annotation_count,api_path,artist_names,full_title,header_image_thumbnail_url,header_image_url,id,language,lyrics_owner_id,lyrics_state,...,primary_artist.api_path,primary_artist.header_image_url,primary_artist.id,primary_artist.image_url,primary_artist.is_meme_verified,primary_artist.is_verified,primary_artist.name,primary_artist.url,stats.pageviews,release_date_components
0,0,/songs/200681,Bob Dylan,"10,000 Men by Bob Dylan",https://images.genius.com/9a0da499b7383ddf2327...,https://images.genius.com/9a0da499b7383ddf2327...,200681,en,225680,complete,...,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
1,0,/songs/7105561,Bob Dylan & The Band,2 Dollars and 99 Cents by Bob Dylan & The Band,https://images.genius.com/84a940f13d290fc51835...,https://images.genius.com/84a940f13d290fc51835...,7105561,en,6190893,complete,...,/artists/369744,https://images.genius.com/a703f2f12d927dd9f1ab...,369744,https://images.genius.com/a703f2f12d927dd9f1ab...,False,False,Bob Dylan & The Band,https://genius.com/artists/Bob-dylan-and-the-band,NaN,NaN
2,1,/songs/200682,Bob Dylan,2 X 2 by Bob Dylan,https://images.genius.com/9a0da499b7383ddf2327...,https://images.genius.com/9a0da499b7383ddf2327...,200682,en,225680,complete,...,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
3,1,/songs/1686914,Bob Dylan,32-20 Blues by Bob Dylan,https://images.genius.com/dec36dae98920363912f...,https://images.genius.com/dec36dae98920363912f...,1686914,en,1549345,complete,...,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
4,15,/songs/105774,Bob Dylan,4th Time Around by Bob Dylan,https://images.genius.com/f96e173addad27a13065...,https://images.genius.com/f96e173addad27a13065...,105774,en,16,complete,...,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,37122.0,NaN
5,1,/songs/994912,Robyn Hitchcock,4th Time Around by Robyn Hitchcock,https://images.genius.com/4476c7b6ab3bd711b40f...,https://images.genius.com/4476c7b6ab3bd711b40f...,994912,en,1549345,complete,...,/artists/90989,https://images.genius.com/75e0332313e5e1a4c076...,90989,https://images.genius.com/75e0332313e5e1a4c076...,False,False,Robyn Hitchcock,https://genius.com/artists/Robyn-hitchcock,NaN,NaN
6,1,/songs/1493543,Traveling Wilburys,7 Deadly Sins by Traveling Wilburys,https://images.genius.com/eda586eb0d4993017d17...,https://images.genius.com/eda586eb0d4993017d17...,1493543,en,1549345,complete,...,/artists/60612,https://images.genius.com/609e88d8b00dca0919f0...,60612,https://images.genius.com/c859d02ba3a25da3b1f9...,False,False,Traveling Wilburys,https://genius.com/artists/Traveling-wilburys,NaN,NaN
7,0,/songs/7103550,Bob Dylan & The Band,900 Miles from My Home by Bob Dylan & The Band,https://images.genius.com/84a940f13d290fc51835...,https://images.genius.com/84a940f13d290fc51835...,7103550,en,6190893,complete,...,/artists/369744,https://images.genius.com/a703f2f12d927dd9f1ab...,369744,https://images.genius.com/a703f2f12d927dd9f1ab...,False,False,Bob Dylan & The Band,https://genius.com/artists/Bob-dylan-and-the-band,NaN,NaN
8,1,/songs/1994655,Bob Dylan,900 Miles from My Home by Bob Dylan,https://images.genius.com/22306423b6ad8777d1ed...,https://images.genius.com/22306423b6ad8777d1ed...,1994655,en,30682,complete,...,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
9,0,/songs/7105812,Bob Dylan & The Band,900 Miles From My Home / Confidential (Hidden ...,https

## Problem 5
Install and import the `lyricsgenius` library in Python, which is a wrapper around `requests` that works specifically with the Genius API. . Follow the guide on the GitHub repository for this library (https://github.com/johnwmillr/LyricsGenius) for instructions on using the library. Use the `lyricsgenius` library to download and display the lyrics to "Tangled Up in Blue" by Bob Dylan. [4 points]

In [10]:
import lyricsgenius
genius = lyricsgenius.Genius(client_access_token)
song = genius.search_song('Tangled Up in Blue')
song.lyrics

Searching for "Tangled Up in Blue"...
Done.


'Tangled Up in Blue Lyrics[Verse 1]\nEarly one morning the sun was shining\nI was laying in bed\nWondering if she\'d changed at all\nIf her hair was still red\nHer folks they said our lives together\nSure was going to be rough\nThey never did like Mama\'s homemade dress\nPapa\'s bankbook wasn\'t big enough\nAnd I was standing on the side of the road\nRain falling on my shoes\nHeading out for the East Coast\nLord knows I\'ve paid some dues\nGetting through\nTangled up in blue\n[Verse 2]\nShe was married when we first met\nSoon to be divorced\nI helped her out of a jam, I guess\nBut I used a little too much force\nWe drove that car as far as we could\nAbandoned it out west\nSplit up on a dark sad night\nBoth agreeing it was best\nShe turned around to look at me\nAs I was walking away\nI heard her say over my shoulder\n"We\'ll meet again someday\nOn the avenue"\nTangled up in blue\n\n[Verse 3]\nI had a job in the great north woods\nWorking as a cook for a spell\nBut I never did like it al